In [ ]:
"""
LINCS 2020数据加载和预处理脚本 - 优化版本 v2
新增智能剂量过滤方法：
- 方法1：百分位数过滤（保留中间80%）
- 方法2：中位数倍数过滤（0.1x-10x）
- 方法3：融合方法
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from numba import jit, prange
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings
import gc
from tqdm import tqdm
warnings.filterwarnings('ignore')

class LINCS2020DataLoader:
    """
    加载和预处理LINCS 2020数据 - 优化版
    修正DOS过滤和时间点匹配，新增智能剂量过滤
    """
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.compound_info = None
        self.inst_info = None  
        self.signatures = None
        self.decompressed_files = []
        
        self.cache_dir = self.data_dir / "_cache"
        self.cache_dir.mkdir(exist_ok=True, parents=True)
    
        print(f"\n{'='*80}")
        print(f"🔬 LINCS 2020 Data Loader - Enhanced Version v2")
        print(f"{'='*80}")
        print(f"Data directory: {self.data_dir}")
        print(f"Dataset: Expanded CMap LINCS Resource 2020")
        print(f"{'='*80}\n")
    
    def load_gene_info(self):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / "geneinfo_beta.txt"
        
        if not gene_file.exists():
            raise FileNotFoundError(
                f"❌ Gene info file not found: {gene_file}\n"
                f"   Please download 'geneinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"📖 Loading gene information...")
        print(f"   File: {gene_file.name}")
        
        gene_info = pd.read_csv(gene_file, sep='\t')
        
        print(f"   ✓ Loaded {len(gene_info):,} genes")
        print(f"   ✓ Columns: {list(gene_info.columns)}")
        
        if 'feature_space' in gene_info.columns:
            landmark_mask = gene_info['feature_space'] == 'landmark'
            landmark_genes = gene_info[landmark_mask].copy()
            print(f"   ✓ Using 'feature_space' column to identify landmarks")
        else:
            raise ValueError(
                f"Cannot identify landmark genes. 'feature_space' column not found.\n"
                f"Available columns: {list(gene_info.columns)}"
            )

        print(f"   ✓ Landmark genes: {len(landmark_genes):,}")
        print(f"   ✓ Expected: 978")

        self.landmark_col_indices = np.where(landmark_mask.values)[0]
        print(f"\n   Landmark column indices (first 10): {self.landmark_col_indices[:10]}")

        self.landmark_gene_ids = set(landmark_genes['gene_id'].astype(str).values)
        self.landmark_gene_symbols = set(landmark_genes['gene_symbol'].astype(str).values)

        print(f"   Sample IDs: {list(self.landmark_gene_ids)[:5]}")
        print(f"   Sample symbols: {list(self.landmark_gene_symbols)[:5]}")

        self.gene_info = gene_info
        return gene_info
    
    def load_cell_info(self):
        """加载细胞系信息"""
        cell_file = self.data_dir / "cellinfo_beta.txt"
        
        if not cell_file.exists():
            raise FileNotFoundError(
                f"❌ Cell info file not found: {cell_file}\n"
                f"   Please download 'cellinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"\n📖 Loading cell information...")
        print(f"   File: {cell_file.name}")
        
        cell_info = pd.read_csv(cell_file, sep='\t')
        
        print(f"   ✓ Loaded {len(cell_info):,} cell lines")
        print(f"   ✓ Columns: {list(cell_info.columns[:10])}...")
        
        if 'cell_iname' in cell_info.columns:
            print(f"   ✓ Unique cell lines (cell_iname): {cell_info['cell_iname'].nunique()}")
        
        print(f"\n   Sample cell lines:")
        for cell in cell_info['cell_iname'].head(5).values:
            print(f"     - {cell}")
        
        self.cell_info = cell_info
        return cell_info
    
    def load_compound_info(self):
        """加载化合物信息"""
        compound_file = self.data_dir / "compoundinfo_beta.txt"
        
        if not compound_file.exists():
            print(f"⚠️  Compound info file not found: {compound_file}")
            print(f"   This file is optional but recommended.")
            return None
        
        print(f"\n📖 Loading compound information...")
        print(f"   File: {compound_file.name}")
        
        compound_info = pd.read_csv(compound_file, sep='\t')
        
        print(f"   ✓ Loaded {len(compound_info):,} compounds")
        print(f"   ✓ Columns: {list(compound_info.columns)}")
        
        if 'pert_id' in compound_info.columns:
            print(f"   ✓ Unique perturbagens: {compound_info['pert_id'].nunique()}")
        
        print(f"\n   Sample compounds:")
        for _, row in compound_info.head(3).iterrows():
            name = row.get('cmap_name', 'Unknown')
            print(f"     - {row['pert_id']}: {name}")
        
        self.compound_info = compound_info
        return compound_info
    
    def load_instance_info(self):
        """加载实例信息"""
        inst_file = self.data_dir / "instinfo_beta.txt"

        if not inst_file.exists():
            raise FileNotFoundError(
                f"❌ Instance info file not found: {inst_file}\n"
                f"   Please download 'instinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )

        print(f"\n📖 Loading instance information...")
        print(f"   File: {inst_file.name}")

        inst_info = pd.read_csv(inst_file, sep='\t')

        print(f"   ✓ Loaded {len(inst_info):,} instances")
        print(f"   ✓ Columns: {list(inst_info.columns[:10])}...")

        if 'inst_id' in inst_info.columns:
            self.instance_join_col = 'inst_id'
        elif 'sample_id' in inst_info.columns:
            self.instance_join_col = 'sample_id'
        else:
            raise ValueError(
                f"Neither 'inst_id' nor 'sample_id' found in instinfo_beta.txt. "
                f"Available columns: {list(inst_info.columns)}"
            )

        print(f"   ✓ Using '{self.instance_join_col}' as join key")

        self.inst_info = inst_info
        return inst_info

    def decompress_gctx_file(self, gctx_file):
        """如果GCTX文件被压缩，则解压"""
        gctx_file = Path(gctx_file)
        
        if not gctx_file.exists():
            raise FileNotFoundError(f"GCTX file not found: {gctx_file}")
        
        if str(gctx_file).endswith('.gz'):
            print(f"⚠️  Detected compressed GCTX file: {gctx_file.name}")
            
            decompressed_dir = self.data_dir / "_decompressed"
            decompressed_dir.mkdir(exist_ok=True)
            
            output_path = decompressed_dir / gctx_file.stem
            
            if output_path.exists():
                print(f"✓ Found existing decompressed file: {output_path.name}")
                return str(output_path)
            
            print(f"📦 Decompressing...")
            with gzip.open(gctx_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            print(f"✓ Decompressed to: {output_path}")
            return str(output_path)
        
        return str(gctx_file)
    
    def read_gctx(self, gctx_file, use_landmark_only=True):
        """读取GCTX文件 - 内存优化版"""
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        file_size_gb = Path(gctx_file).stat().st_size / (1024**3)
        print(f"   File size: {file_size_gb:.2f} GB")
        
        if file_size_gb > 50:
            print(f"   ⚠️  Large file detected. Using memory-optimized loading...")
        
        gctx_file = self.decompress_gctx_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            print(f"📊 Inspecting HDF5 structure...")
            print(f"   Available keys: {list(f.keys())}")
            
            if '/0/DATA/0/matrix' in f:
                matrix_dataset = f['/0/DATA/0/matrix']
                row_path = '/0/META/ROW'
                col_path = '/0/META/COL'
            elif '/matrix' in f:
                matrix_dataset = f['/matrix']
                row_path = '/row'
                col_path = '/col'
            else:
                raise ValueError(f"Cannot find matrix in GCTX file. Available keys: {list(f.keys())}")
            
            matrix_shape = matrix_dataset.shape
            print(f"   ✓ Matrix shape: {matrix_shape} (samples × genes)")
            
            print(f"\n📋 Loading metadata...")
            
            # 读取样本元数据
            sample_meta = {}
            if row_path in f:
                print(f"   Reading sample metadata from: {row_path}")
                print(f"   Available row fields: {list(f[row_path].keys())}")
                
                for key in f[row_path].keys():
                    data = f[f'{row_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            sample_meta[key] = data.astype(str)
                    else:
                        sample_meta[key] = data
                
                print(f"   ✓ Loaded {len(sample_meta)} sample metadata fields")
            else:
                raise ValueError(f"Cannot find row metadata at: {row_path}")
            
            # 读取基因元数据
            gene_meta = {}
            if col_path in f:
                print(f"   Reading gene metadata from: {col_path}")
                print(f"   Available col fields: {list(f[col_path].keys())}")
                
                for key in f[col_path].keys():
                    data = f[f'{col_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            gene_meta[key] = data.astype(str)
                    else:
                        gene_meta[key] = data
                
                print(f"   ✓ Loaded {len(gene_meta)} gene metadata fields")
            else:
                raise ValueError(f"Cannot find col metadata at: {col_path}")
            
            sample_df = pd.DataFrame(sample_meta)
            gene_df = pd.DataFrame(gene_meta)

            # 自动检测并交换ROW/COL
            if self.gene_info is not None:
                n_features_expected = len(self.gene_info)
                if len(sample_df) == n_features_expected and len(gene_df) != n_features_expected:
                    print(f"\n   ⚠️ Detected ROW/COL swap. Correcting...")
                    sample_df, gene_df = gene_df, sample_df

            print(f"\n   Sample metadata columns: {list(sample_df.columns[:10])}...")
            print(f"   Gene metadata columns: {list(gene_df.columns)}")
            
            # 确定landmark基因的列索引
            landmark_col_indices = None
            if use_landmark_only:
                print(f"\n🔬 Filtering to landmark genes...")

                if (self.gene_info is None) or (not hasattr(self, "landmark_col_indices")):
                    print(f"   Loading gene info to get landmark indices...")
                    self.load_gene_info()

                if len(self.gene_info) != matrix_shape[1]:
                    print(f"   ⚠️  Warning: geneinfo rows ({len(self.gene_info)}) "
                        f"!= GCTX feature count ({matrix_shape[1]})")
                
                landmark_col_indices = np.array(self.landmark_col_indices, dtype=int)
                print(f"   ✓ Using {len(landmark_col_indices)} landmark features "
                    f"out of {matrix_shape[1]} total")
            
            # 内存优化：分块读取
            print(f"\n🎯 Loading data (memory-optimized)...")
            if landmark_col_indices is not None:
                print(f"   Reading {len(landmark_col_indices)} columns out of {matrix_shape[1]}...")
                chunk_size = 610000
                chunks = []
                
                for start_idx in range(0, matrix_shape[0], chunk_size):
                    end_idx = min(start_idx + chunk_size, matrix_shape[0])
                    print(f"   Loading rows {start_idx:,} to {end_idx:,}... ({end_idx/matrix_shape[0]*100:.1f}%)", end='\r')
                    
                    chunk = matrix_dataset[start_idx:end_idx, landmark_col_indices].astype(np.float32)
                    chunks.append(chunk)
                    
                    if len(chunks) >= 10:
                        print(f"\n   Consolidating chunks...")
                        merged = np.vstack(chunks)
                        chunks = [merged]
                        gc.collect()
                
                print(f"\n   Finalizing matrix...")
                matrix = np.vstack(chunks) if len(chunks) > 1 else chunks[0]
                del chunks
                gc.collect()
            else:
                print(f"   Reading full matrix...")
                matrix = matrix_dataset[:].astype(np.float32)
            
            print(f"   ✓ Final matrix shape: {matrix.shape}")
            print(f"   ✓ Memory usage: {matrix.nbytes / (1024**3):.2f} GB")
            print(f"   ✓ Data type: {matrix.dtype}")
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self):
        """加载Level 4数据"""
        level4_file = self.data_dir / "level4_beta_trt_cp_n1805898x12328.gctx"
        
        if not level4_file.exists():
            pattern = self.data_dir / "level4_beta_trt_cp*.gctx"
            files = glob.glob(str(pattern))
            
            if not files:
                raise FileNotFoundError(
                    f"❌ Level 4 file not found: {level4_file}\n"
                    f"   Please download from: https://clue.io/data/CMap2020#LINCS2020"
                )
            
            level4_file = files[0]
        
        print(f"\n{'='*80}")
        print(f"📖 Loading Level 4 Signatures")
        print(f"{'='*80}")
        print(f"File: {level4_file.name}")
        
        matrix, sample_meta, gene_meta = self.read_gctx(
            level4_file, 
            use_landmark_only=True
        )

        # 用instinfo补齐metadata
        if 'id' in sample_meta.columns:
            sample_meta = sample_meta.rename(columns={'id': 'sample_id'})
        elif 'sample_id' not in sample_meta.columns:
            raise ValueError("Cannot find 'id' or 'sample_id' in GCTX ROW metadata")

        if self.inst_info is None:
            self.load_instance_info()

        inst_info = self.inst_info
        join_col = getattr(self, "instance_join_col", None)
        
        if join_col is None:
            join_col = 'sample_id' if 'sample_id' in inst_info.columns else 'inst_id'

        if join_col not in sample_meta.columns:
            if join_col == 'inst_id' and 'sample_id' in sample_meta.columns:
                print("   ⚠️ Using 'sample_id' instead of 'inst_id'")
                join_col = 'sample_id'
            else:
                raise ValueError(
                    f"Join column '{join_col}' not found in GCTX metadata. "
                    f"Available: {list(sample_meta.columns)}"
                )

        print(f"   🔗 Merging GCTX metadata with instinfo on '{join_col}'...")
        merged_meta = sample_meta.merge(inst_info, on=join_col, how='left')

        if 'pert_id' not in merged_meta.columns:
            raise ValueError("After merging, 'pert_id' is still missing")

        n_missing = merged_meta['pert_id'].isna().sum()
        if n_missing > 0:
            print(f"   ⚠️ {n_missing:,} rows have missing pert_id")

        self.signatures = {
            'matrix': matrix,
            'row_meta': merged_meta,
            'col_meta': gene_meta
        }

        return matrix, merged_meta, gene_meta
    
    def calculate_cosine_similarity_to_nearest_replicate(
        self, 
        matrix: np.ndarray, 
        pert_ids: pd.Series
    ) -> np.ndarray:
        """
        使用Numba JIT编译的超高效版本
        
        计算每个样本与其最近同化合物复制品的余弦相似度
        """

        print(f"\n📊 Calculating similarities (Numba-accelerated)...")
        
        # 预归一化
        matrix_norm = normalize(matrix, norm='l2', axis=1).astype(np.float32)
        
        @jit(nopython=True, parallel=True, fastmath=True)
        def compute_max_similarities(data, indices, n_total):
            """Numba加速的核心计算"""
            n = len(indices)
            result = np.full(n_total, -np.inf, dtype=np.float32)
            
            for i in prange(n):
                idx_i = indices[i]
                vec_i = data[i]
                max_sim = -np.inf
                
                for j in range(n):
                    if i != j:
                        # 点积（已归一化 = 余弦相似度）
                        sim = np.dot(vec_i, data[j])
                        if sim > max_sim:
                            max_sim = sim
                
                result[idx_i] = max_sim
            
            return result
        
        nearest_similarities = np.zeros(len(pert_ids), dtype=np.float32)
        
        # 按化合物分组
        pert_ids_array = pert_ids.values
        unique_perts = np.unique(pert_ids_array)
        
        from tqdm import tqdm
        for pert_id in tqdm(unique_perts, desc="   Computing"):
            mask = pert_ids_array == pert_id
            indices = np.where(mask)[0]
            
            if len(indices) < 2:
                nearest_similarities[indices] = 0.0
                continue
            
            pert_data = matrix_norm[mask]
            sims = compute_max_similarities(pert_data, indices, len(pert_ids))
            nearest_similarities[indices] = sims[indices]
        
        print(f"   ✓ Mean similarity: {nearest_similarities.mean():.4f}")
        return nearest_similarities
    
    def dose_filter_method1(self, row_meta, valid_mask, percentile_range=(10, 90)):
        """
        方法1：百分位数过滤 - 向量化优化版本
        
        性能提升：使用groupby + rank进行批量百分位计算，避免循环
        """
        print(f"\n   📊 Method 1: Percentile-based filtering ({percentile_range[0]}th-{percentile_range[1]}th percentile)")
        print(f"      Using vectorized operations for {row_meta[valid_mask]['pert_id'].nunique():,} compounds...")
        
        # 创建工作DataFrame（只包含有效样本）
        work_df = row_meta[valid_mask][['pert_id', 'dose_uM']].copy()
        work_df = work_df[work_df['dose_uM'].notna()].copy()
        
        if len(work_df) == 0:
            print(f"      ⚠️  No valid dose data")
            return valid_mask
        
        # 批量计算每个样本在其化合物内的百分位排名
        work_df['dose_rank_pct'] = work_df.groupby('pert_id')['dose_uM'].rank(pct=True) * 100
        
        # 一次性标记所有在范围内的样本
        in_percentile_range = (work_df['dose_rank_pct'] >= percentile_range[0]) & \
                            (work_df['dose_rank_pct'] <= percentile_range[1])
        
        # 向量化计算每个剂量值的"大部分在内/在外"判断
        # 对于每个 (compound, dose) 组合，统计有多少在范围内
        dose_groups = work_df.groupby(['pert_id', 'dose_uM']).agg({
            'dose_rank_pct': ['count']  # 总数
        }).reset_index()
        dose_groups.columns = ['pert_id', 'dose_uM', 'total_count']
        
        # 统计在范围内的数量
        in_range_counts = work_df[in_percentile_range].groupby(['pert_id', 'dose_uM']).size().reset_index(name='in_range_count')
        dose_groups = dose_groups.merge(in_range_counts, on=['pert_id', 'dose_uM'], how='left')
        dose_groups['in_range_count'] = dose_groups['in_range_count'].fillna(0)
        dose_groups['out_range_count'] = dose_groups['total_count'] - dose_groups['in_range_count']
        
        # 大部分在范围内的剂量值保留
        dose_groups['keep'] = dose_groups['in_range_count'] > dose_groups['out_range_count']
        
        # 创建 (compound, dose) -> keep 的映射
        keep_doses = dose_groups[dose_groups['keep']][['pert_id', 'dose_uM']]
        keep_doses['_keep_flag'] = True
        
        # 批量合并判断结果
        work_df = work_df.merge(keep_doses, on=['pert_id', 'dose_uM'], how='left')
        work_df['_keep_flag'] = work_df['_keep_flag'].fillna(False)
        
        # 构建最终mask
        dose_mask = np.zeros(len(row_meta), dtype=bool)
        dose_mask[valid_mask] = True
        
        # 将不保留的样本标记为False
        valid_indices = np.where(valid_mask)[0]
        valid_dose_indices = work_df.index
        remove_indices = valid_dose_indices[~work_df['_keep_flag'].values]
        dose_mask[remove_indices] = False
        
        n_removed = valid_mask.sum() - dose_mask.sum()
        print(f"      ✓ Removed {n_removed:,} samples")
        print(f"      ✓ Remaining: {dose_mask.sum():,} samples")
        
        return dose_mask
    
    def dose_filter_method2(self, row_meta, valid_mask, fold_range=(0.1, 10), central_measure='median'):
        """
        方法2：中位数/众数倍数过滤 - 向量化优化版本
        
        性能提升：使用groupby.transform()批量计算，一次性完成所有判断
        """
        print(f"\n   📊 Method 2: Central value fold-range filtering ({fold_range[0]}x-{fold_range[1]}x, using {central_measure})")
        print(f"      Using vectorized operations for {row_meta[valid_mask]['pert_id'].nunique():,} compounds...")
        
        # 创建工作DataFrame
        work_df = row_meta[valid_mask][['pert_id', 'dose_uM']].copy()
        work_df = work_df[work_df['dose_uM'].notna()].copy()
        
        if len(work_df) == 0:
            print(f"      ⚠️  No valid dose data")
            return valid_mask
        
        # 批量计算每个化合物的中心值（一次性完成）
        if central_measure == 'median':
            # 使用transform直接将中位数映射到每一行
            work_df['central_value'] = work_df.groupby('pert_id')['dose_uM'].transform('median')
        elif central_measure == 'mode':
            # 众数需要特殊处理（众数可能不唯一，取第一个）
            def first_mode(x):
                mode_vals = x.mode()
                return mode_vals[0] if len(mode_vals) > 0 else x.median()
            
            work_df['central_value'] = work_df.groupby('pert_id')['dose_uM'].transform(first_mode)
        else:
            work_df['central_value'] = work_df.groupby('pert_id')['dose_uM'].transform('median')
        
        # 向量化计算上下界
        work_df['lower_bound'] = work_df['central_value'] * fold_range[0]
        work_df['upper_bound'] = work_df['central_value'] * fold_range[1]
        
        # 向量化判断所有样本是否在范围内
        work_df['keep'] = (work_df['dose_uM'] >= work_df['lower_bound']) & \
                        (work_df['dose_uM'] <= work_df['upper_bound'])
        
        # 构建最终mask
        dose_mask = np.zeros(len(row_meta), dtype=bool)
        dose_mask[valid_mask] = True
        
        # 标记不保留的样本
        remove_indices = work_df.index[~work_df['keep'].values]
        dose_mask[remove_indices] = False
        
        n_removed = valid_mask.sum() - dose_mask.sum()
        print(f"      ✓ Removed {n_removed:,} samples")
        print(f"      ✓ Remaining: {dose_mask.sum():,} samples")
        
        return dose_mask
    
    def dose_filter_method3(self, row_meta, valid_mask, percentile_range=(10, 90), 
                           fold_range=(0.1, 10), central_measure='median'):
        """
        方法3：融合方法1和方法2
        
        同时满足百分位数条件和倍数条件的样本才保留（取交集）
        
        Parameters:
        -----------
        row_meta : pd.DataFrame
            包含 'pert_id' 和 'dose_uM' 列的元数据
        valid_mask : np.ndarray
            当前有效样本的mask
        percentile_range : tuple
            方法1的百分位数范围
        fold_range : tuple
            方法2的倍数范围
        central_measure : str
            方法2使用的中心度量
        
        Returns:
        --------
        np.ndarray : 更新后的valid_mask
        """
        print(f"\n   📊 Method 3: Hybrid filtering (Method 1 AND Method 2)")
        
        # 应用方法1
        mask1 = self.dose_filter_method1(row_meta, valid_mask, percentile_range)
        
        # 应用方法2
        mask2 = self.dose_filter_method2(row_meta, valid_mask, fold_range, central_measure)
        
        # 取交集
        hybrid_mask = mask1 & mask2
        
        n_removed_method1 = valid_mask.sum() - mask1.sum()
        n_removed_method2 = valid_mask.sum() - mask2.sum()
        n_removed_hybrid = valid_mask.sum() - hybrid_mask.sum()
        
        print(f"\n      Summary:")
        print(f"      - Method 1 alone would remove: {n_removed_method1:,} samples")
        print(f"      - Method 2 alone would remove: {n_removed_method2:,} samples")
        print(f"      - Hybrid method removes: {n_removed_hybrid:,} samples")
        
        return hybrid_mask
    
    def prepare_training_data(
        self, 
        min_observations_per_compound=5,
        min_replicate_similarity=0.12,
        dose_filter_method='method1',  # 'method1', 'method2', 'method3', or None
        dose_percentile_range=(10, 90),
        dose_fold_range=(0.1, 10),
        dose_central_measure='median',
        valid_timepoints=[6, 24],
        min_cell_lines=5,
        max_cell_lines=200,
        min_compounds_per_cell=200,
        remove_dos=True,
        enable_visualization=False
    ):
        """
        准备训练数据
        
        Parameters:
        -----------
        dose_filter_method : str or None
            剂量过滤方法: 'method1', 'method2', 'method3', 或 None (不过滤剂量)
        dose_percentile_range : tuple
            方法1的百分位数范围，例如 (10, 90) 保留中间80%
        dose_fold_range : tuple
            方法2的倍数范围，例如 (0.1, 10)
        dose_central_measure : str
            方法2使用的中心度量: 'median' 或 'mode'
        enable_visualization : bool
            是否生成可视化图表
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta'].copy()
        col_meta = self.signatures['col_meta']
        
        print(f"\n[DEBUG] row_meta shape: {row_meta.shape}")
        print(f"[DEBUG] columns (first 15): {list(row_meta.columns[:15])}")
        print(f"[DEBUG] Example row:")
        print(row_meta.iloc[0][['sample_id', 'pert_id', 'pert_type', 'cell_iname', 'pert_time', 'pert_dose']])

        print("\n" + "=" * 80)
        print("🔍 DRUGREFLECTOR QUALITY CONTROL PIPELINE - Enhanced v2")
        print("=" * 80)
        print(f"Initial samples: {len(row_meta):,}")
        print(f"Initial memory: {matrix.nbytes / (1024**3):.2f} GB")
        
        print(f"\n📋 Available metadata columns:")
        for col in row_meta.columns[:15]:
            sample_val = row_meta[col].iloc[0] if len(row_meta) > 0 else 'N/A'
            print(f"   - {col}: {sample_val}")
        if len(row_meta.columns) > 15:
            print(f"   ... and {len(row_meta.columns) - 15} more")
        
        # 检查必需字段
        required_fields = ['pert_id']
        missing_fields = [f for f in required_fields if f not in row_meta.columns]
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        # 确定细胞系ID字段
        cell_id_col = None
        for possible_col in ['cell_id', 'cell_iname', 'cell_mfc_name']:
            if possible_col in row_meta.columns:
                cell_id_col = possible_col
                print(f"\n✓ Using '{cell_id_col}' as cell line identifier")
                break
        
        if cell_id_col is None:
            print(f"\n⚠️  Warning: Cannot find cell line identifier")
        
        valid_mask = np.ones(len(row_meta), dtype=bool)
        
        initial_compounds = row_meta['pert_id'].nunique()
        print(f"\nInitial compounds: {initial_compounds:,}")
        
        # ========== Filter 1: Remove DOS compounds ==========
        if remove_dos:
            print(f"\n{'='*80}")
            print(f"FILTER 1: Remove DOS compounds (keep trt_cp only)")
            print(f"{'='*80}")
            
            if 'pert_type' in row_meta.columns:
                print(f"   pert_type value counts:")
                pert_type_counts = row_meta['pert_type'].value_counts()
                for ptype, count in pert_type_counts.items():
                    print(f"     - {ptype}: {count:,} samples")
                
                dos_mask = row_meta['pert_type'] == 'trt_cp'
                n_removed = (~dos_mask).sum()
                
                print(f"\n  ✓ Keeping only 'trt_cp' perturbations")
                print(f"  Removed {n_removed:,} non-trt_cp observations")
            else:
                print(f"   ⚠️ 'pert_type' not found, using fallback method")
                dos_mask = ~row_meta['pert_id'].str.contains('DOS', case=False, na=False)
                n_removed = (~dos_mask).sum()
                print(f"  Removed {n_removed:,} DOS observations")
            
            valid_mask &= dos_mask
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        # ========== Filter 2: Minimum observations ==========
        print(f"\n{'='*80}")
        print(f"FILTER 2: Remove compounds with <{min_observations_per_compound} observations")
        print(f"{'='*80}")
        
        valid_meta = row_meta[valid_mask]
        obs_counts = valid_meta.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations_per_compound].index
        
        print(f"  Compounds with ≥{min_observations_per_compound} observations: "
              f"{len(valid_perts):,}/{obs_counts.nunique():,}")
        
        obs_mask = row_meta['pert_id'].isin(valid_perts)
        valid_mask &= obs_mask
        
        print(f"  Remaining samples: {valid_mask.sum():,}")
        print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        del valid_meta, obs_counts, obs_mask
        gc.collect()
            
        # ========== Filter 3: Cosine similarity ==========
        print(f"\n{'='*80}")
        print(f"FILTER 3: Remove observations with cosine similarity <{min_replicate_similarity}")
        print(f"{'='*80}")
        
        valid_indices = np.where(valid_mask)[0]
        valid_matrix = matrix[valid_mask]
        valid_pert_ids = row_meta.loc[valid_mask, 'pert_id'].reset_index(drop=True)
        
        nearest_similarities = self.calculate_cosine_similarity_to_nearest_replicate(
            valid_matrix, 
            valid_pert_ids
        )
        
        full_similarities = np.zeros(len(row_meta), dtype=np.float32)
        full_similarities[valid_indices] = nearest_similarities
        
        sim_mask = (full_similarities >= min_replicate_similarity) | (~valid_mask)
        n_removed_sim = (~sim_mask & valid_mask).sum()
        valid_mask &= sim_mask
        
        print(f"  Removed {n_removed_sim:,} low-similarity observations")
        print(f"  Remaining samples: {valid_mask.sum():,}")
        
        del valid_matrix, valid_pert_ids, nearest_similarities, full_similarities, sim_mask
        gc.collect()
        
        # ========== Filter 4: Dose selection (新方法) ==========
        print(f"\n{'='*80}")
        print(f"FILTER 4: Intelligent Dose Filtering")
        print(f"{'='*80}")

        if 'pert_dose' in row_meta.columns and dose_filter_method is not None:
            
            # Step 1: 解析剂量值
            print(f"  Step 1/3: Parsing dose values...")
            dose_str = row_meta['pert_dose'].astype(str)
            row_meta['dose_value'] = pd.to_numeric(
                dose_str.str.replace(r'[^\d.]', '', regex=True), 
                errors='coerce'
            )
            row_meta['dose_unit'] = dose_str.str.extract(r'([a-zA-Z]+)', expand=False).str.lower()
            
            valid_dose_count = row_meta['dose_value'].notna().sum()
            invalid_dose_count = row_meta['dose_value'].isna().sum()
            print(f"  ✓ Successfully parsed: {valid_dose_count:,} entries")
            print(f"  ⚠️  Missing/invalid doses: {invalid_dose_count:,} entries")
            
            # Step 2: 转换单位到 µM
            print(f"\n  Step 2/3: Converting to µM...")
            dose_value = row_meta['dose_value'].values
            dose_unit = row_meta['dose_unit'].values
            
            conditions = [
                dose_unit == 'nm',
                dose_unit == 'mm'
            ]
            choices = [
                dose_value / 1000,
                dose_value * 1000
            ]
            row_meta['dose_uM'] = np.select(conditions, choices, default=dose_value)
            
            print(f"  Dose unit distribution:")
            unit_counts = row_meta['dose_unit'].value_counts()
            for unit, count in unit_counts.head(5).items():
                print(f"    - {unit}: {count:,} samples")
            
            # Step 3: 应用选定的剂量过滤方法
            print(f"\n  Step 3/3: Applying dose filter (Method: {dose_filter_method})...")
            
            n_before_dose = valid_mask.sum()
            
            if dose_filter_method == 'method1':
                valid_mask = self.dose_filter_method1(
                    row_meta, valid_mask, dose_percentile_range
                )
            elif dose_filter_method == 'method2':
                valid_mask = self.dose_filter_method2(
                    row_meta, valid_mask, dose_fold_range, dose_central_measure
                )
            elif dose_filter_method == 'method3':
                valid_mask = self.dose_filter_method3(
                    row_meta, valid_mask, dose_percentile_range, 
                    dose_fold_range, dose_central_measure
                )
            else:
                print(f"  ⚠️  Unknown method '{dose_filter_method}', skipping dose filter")
            
            n_removed_dose = n_before_dose - valid_mask.sum()
            
            print(f"\n  ✅ Filter 4 Results:")
            print(f"  ✓ Removed {n_removed_dose:,} samples")
            print(f"  ✓ Remaining samples: {valid_mask.sum():,}")
            print(f"  ✓ Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            # 可视化（如果启用）
            if enable_visualization:
                self._visualize_dose_filtering(row_meta, valid_mask, valid_indices)
                
        elif dose_filter_method is None:
            print(f"  ℹ️  Dose filtering disabled (dose_filter_method=None)")
        else:
            print(f"  ⚠️  'pert_dose' not found, skipping dose filter")
        
        # ========== Filter 5: Timepoint selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 5: Keep only measurements at {valid_timepoints} hours")
        print(f"{'='*80}")
        
        if 'pert_time' in row_meta.columns:
            row_meta['time_numeric'] = pd.to_numeric(row_meta['pert_time'], errors='coerce')
            
            print(f"   Available timepoints (numeric):")
            time_counts = row_meta['time_numeric'].value_counts().head(10)
            for time_val, count in time_counts.items():
                print(f"     - {time_val} hours: {count:,} samples")
            
            time_mask = row_meta['time_numeric'].isin(valid_timepoints)
            
            if time_mask.sum() == 0:
                print(f"  ⚠️  No samples match timepoints {valid_timepoints}")
                print(f"  Skipping timepoint filter...")
            else:
                n_before = valid_mask.sum()
                valid_mask &= time_mask
                n_removed = n_before - valid_mask.sum()
                
                print(f"  ✓ Kept samples at {valid_timepoints} hours")
                print(f"  Removed {n_removed:,} observations (invalid timepoint)")
                print(f"  Remaining samples: {valid_mask.sum():,}")
            
            del time_mask
            gc.collect()
        else:
            print(f"  ⚠️  'pert_time' not found, skipping timepoint filter")

        # ========== Filter 6: Remove cell lines with <N compounds ==========
        print(f"\n{'='*80}")
        print(f"FILTER 6: Remove cell lines with <{min_compounds_per_cell} compounds")
        print(f"{'='*80}")

        if cell_id_col is not None:
            valid_meta = row_meta[valid_mask]
            
            compounds_per_cell = valid_meta.groupby(cell_id_col)['pert_id'].nunique()
            
            print(f"  Total cell lines before filtering: {len(compounds_per_cell):,}")
            print(f"  Compounds per cell line (mean): {compounds_per_cell.mean():.1f}")
            print(f"  Compounds per cell line (median): {compounds_per_cell.median():.0f}")
            print(f"  Compounds per cell line (min): {compounds_per_cell.min():.0f}")
            print(f"  Compounds per cell line (max): {compounds_per_cell.max():.0f}")
            
            valid_cells = compounds_per_cell[compounds_per_cell >= min_compounds_per_cell].index
            
            print(f"\n  Cell lines with ≥{min_compounds_per_cell} compounds: "
                f"{len(valid_cells):,}/{len(compounds_per_cell):,}")
            
            removed_cells = compounds_per_cell[compounds_per_cell < min_compounds_per_cell]
            if len(removed_cells) > 0:
                print(f"\n  Sample removed cell lines (showing up to 10):")
                for cell, n_compounds in removed_cells.head(10).items():
                    print(f"    - {cell}: {n_compounds} compounds")
            
            cell_filter_mask = row_meta[cell_id_col].isin(valid_cells)
            n_before = valid_mask.sum()
            valid_mask &= cell_filter_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"\n  Removed {n_removed:,} observations from {len(removed_cells):,} cell lines")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            print(f"  Remaining cell lines: {len(valid_cells):,}")
            
            del valid_meta, compounds_per_cell, cell_filter_mask, removed_cells
            gc.collect()
        else:
            print(f"  ⚠️  Cell line column not found, skipping filter")
        
        # ========== Filter 7: Cell line count ==========
        print(f"\n{'='*80}")
        print(f"FILTER 7: Remove compounds in <{min_cell_lines} or >{max_cell_lines} cell lines")
        print(f"{'='*80}")
        
        if cell_id_col is not None:
            valid_meta = row_meta[valid_mask]
            cell_line_counts = valid_meta.groupby('pert_id')[cell_id_col].nunique()
            valid_perts_cell = cell_line_counts[
                (cell_line_counts >= min_cell_lines) & 
                (cell_line_counts <= max_cell_lines)
            ].index
            
            print(f"  Compounds in {min_cell_lines}-{max_cell_lines} cell lines: "
                  f"{len(valid_perts_cell):,}/{len(cell_line_counts):,}")
            
            cell_mask = row_meta['pert_id'].isin(valid_perts_cell)
            n_before = valid_mask.sum()
            valid_mask &= cell_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"  Removed {n_removed:,} observations")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            del valid_meta, cell_line_counts, cell_mask
            gc.collect()
        else:
            print(f"  ⚠️  Cell line column not found, skipping")
        
        # ========== 最终提取数据 ==========
        print(f"\n{'='*80}")
        print(f"✅ FINAL DATASET")
        print(f"{'='*80}")
        
        final_matrix = matrix[valid_mask].copy()
        final_meta = row_meta[valid_mask].reset_index(drop=True)
        
        del matrix
        gc.collect()
        
        print(f"  Extracted {len(final_matrix):,} samples")
        print(f"  Memory usage: {final_matrix.nbytes / (1024**3):.2f} GB")
        
        # 创建标签
        unique_perts = sorted(final_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in final_meta['pert_id']], dtype=np.int32)
        
        final_compounds = len(unique_perts)
        final_samples = len(final_matrix)
        
        if cell_id_col:
            final_cells = final_meta[cell_id_col].nunique()
        else:
            final_cells = 'Unknown'
        
        print(f"\n  Total samples: {final_samples:,}")
        print(f"  Total compounds: {final_compounds:,}")
        print(f"  Cell lines: {final_cells}")
        print(f"  Gene features: {final_matrix.shape[1]}")
        
        compound_obs = final_meta.groupby('pert_id').size()
        print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
        print(f"  Samples per compound (median): {compound_obs.median():.0f}")
        
        n_compounds_100plus = (compound_obs > 100).sum()
        print(f"  Compounds with >100 observations: {n_compounds_100plus:,}")
        
        print(f"\n📊 Comparison with paper (SI page 2):")
        print(f"  Paper: 425,242 obs, 9,597 compounds, 52 cell lines")
        print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cell lines")
        
        if initial_compounds > 0:
            retention_rate = (final_compounds / initial_compounds) * 100
            print(f"  Compound retention rate: {retention_rate:.1f}%")
        
        # 基因名
        if (self.gene_info is not None) and hasattr(self, "landmark_col_indices"):
            gi = self.gene_info
            landmark_mask = gi['feature_space'] == 'landmark'
            landmark_geneinfo = gi[landmark_mask]
            gene_names = list(landmark_geneinfo['gene_symbol'].values)
            print(f"  ✓ Using geneinfo_beta.txt for gene names")
        else:
            gene_name_col = None
            for possible_col in ['gene_symbol', 'pr_gene_symbol', 'symbol', 'id']:
                if possible_col in col_meta.columns:
                    gene_name_col = possible_col
                    break
            if gene_name_col is None:
                gene_name_col = col_meta.columns[0]
            gene_names = list(col_meta[gene_name_col].values)
            print(f"  ⚠️ Using '{gene_name_col}' from col_meta for gene names")

        training_data = {
            'X': final_matrix,
            'y': labels,
            'folds': np.zeros(len(final_matrix), dtype=np.int32),
            'sample_meta': final_meta,
            'metadata': final_meta,
            'gene_names': gene_names,
            'compound_names': list(unique_perts),
            'pert_to_idx': pert_to_idx
        }

        gc.collect()
        
        return training_data
    
    def _visualize_dose_filtering(self, row_meta, valid_mask, valid_indices):
        """生成剂量过滤的可视化图表"""
        print(f"\n  📊 Generating dose distribution visualizations...")
        
        viz_dir = Path("E:/科研/Models/drugreflector/visualizations")
        viz_dir.mkdir(exist_ok=True, parents=True)
        
        # 确保valid_indices存在且有效
        if not hasattr(self, 'valid_indices') or valid_indices is None:
            valid_indices = np.where(valid_mask)[0]
        
        # 获取过滤前后的剂量数据
        doses_before = row_meta.loc[valid_indices, 'dose_uM'].dropna()
        doses_after = row_meta.loc[valid_mask, 'dose_uM'].dropna()
        
        if len(doses_before) == 0 or len(doses_after) == 0:
            print(f"  ⚠️  Insufficient dose data for visualization")
            return
        
        # ===== 图1: 剂量分布对比 =====
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # 子图1: 过滤前
        ax1 = axes[0]
        log_bins = np.logspace(np.log10(max(0.001, doses_before.min())), 
                               np.log10(doses_before.max()), 50)
        ax1.hist(doses_before, bins=log_bins, edgecolor='black', alpha=0.7, color='steelblue')
        ax1.set_xscale('log')
        ax1.set_xlabel('Dose (µM, log scale)', fontsize=11)
        ax1.set_ylabel('Number of samples', fontsize=11)
        ax1.set_title('Dose Distribution Before Filtering', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        # 子图2: 过滤后
        ax2 = axes[1]
        log_bins = np.logspace(np.log10(max(0.001, doses_after.min())), 
                               np.log10(doses_after.max()), 50)
        ax2.hist(doses_after, bins=log_bins, edgecolor='black', alpha=0.7, color='forestgreen')
        ax2.set_xscale('log')
        ax2.set_xlabel('Dose (µM, log scale)', fontsize=11)
        ax2.set_ylabel('Number of samples', fontsize=11)
        ax2.set_title('Dose Distribution After Filtering', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        output_path = viz_dir / "dose_distribution_comparison.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"  ✓ Saved figure: {output_path}")
        plt.close()
        
        # ===== 图2: Top剂量值统计 =====
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # 子图1: 过滤前Top 15
        ax1 = axes[0]
        top_doses_before = doses_before.value_counts().head(15)
        bars1 = ax1.barh(range(len(top_doses_before)), top_doses_before.values, 
                         color='steelblue', edgecolor='black', alpha=0.8)
        ax1.set_yticks(range(len(top_doses_before)))
        ax1.set_yticklabels([f'{d:.2f} µM' for d in top_doses_before.index])
        ax1.set_xlabel('Sample Count', fontsize=11)
        ax1.set_ylabel('Dose', fontsize=11)
        ax1.set_title('Top 15 Doses Before Filtering', fontsize=12, fontweight='bold')
        ax1.grid(axis='x', alpha=0.3)
        
        for i, val in enumerate(top_doses_before.values):
            ax1.text(val, i, f' {val:,}', va='center', fontsize=8)
        
        # 子图2: 过滤后Top 15
        ax2 = axes[1]
        top_doses_after = doses_after.value_counts().head(15)
        bars2 = ax2.barh(range(len(top_doses_after)), top_doses_after.values, 
                         color='forestgreen', edgecolor='black', alpha=0.8)
        ax2.set_yticks(range(len(top_doses_after)))
        ax2.set_yticklabels([f'{d:.2f} µM' for d in top_doses_after.index])
        ax2.set_xlabel('Sample Count', fontsize=11)
        ax2.set_ylabel('Dose', fontsize=11)
        ax2.set_title('Top 15 Doses After Filtering', fontsize=12, fontweight='bold')
        ax2.grid(axis='x', alpha=0.3)
        
        for i, val in enumerate(top_doses_after.values):
            ax2.text(val, i, f' {val:,}', va='center', fontsize=8)
        
        plt.tight_layout()
        output_path2 = viz_dir / "top_doses_comparison.png"
        plt.savefig(output_path2, dpi=300, bbox_inches='tight')
        print(f"  ✓ Saved figure: {output_path2}")
        plt.close()
        
        print(f"\n  ✅ Visualization complete!")
        print(f"     Figures saved to: {viz_dir}")
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*80}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*80}")
        
        folds = np.zeros(len(sample_meta), dtype=np.int32)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


def main():
    """主程序 - 优化版 v2"""
    print("=" * 80)
    print("🧬 DRUGREFLECTOR DATA PREPROCESSING - Enhanced Version v2")
    print("   With intelligent dose filtering methods")
    print("=" * 80)
    
    data_dir = "E:/科研/Models/drugreflector/datasets/LINCS2020"
    loader = LINCS2020DataLoader(data_dir)
    
    try:
        # Step 1: 加载元数据
        print("\n" + "=" * 80)
        print("STEP 1: Loading metadata")
        print("=" * 80)
        gene_info = loader.load_gene_info()
        cell_info = loader.load_cell_info()
        compound_info = loader.load_compound_info()
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 80)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 80)
        matrix, row_meta, col_meta = loader.load_level4_signatures()
        
        # Step 3: 准备训练数据
        print("\n" + "=" * 80)
        print("STEP 3: Preparing training data")
        print("=" * 80)
        training_data = loader.prepare_training_data(
            min_observations_per_compound=5,
            min_replicate_similarity=0.12,
            dose_filter_method='method2',  # 'method1', 'method2', 'method3', or None
            dose_percentile_range=(10, 90),  # 保留中间80%
            dose_fold_range=(0.2, 50),       # 0.1x到10x
            dose_central_measure='median',   # 使用中位数
            valid_timepoints=[6, 24],
            min_cell_lines=5,
            max_cell_lines=200,
            min_compounds_per_cell=300,
            remove_dos=True,
            enable_visualization=True  # 启用可视化
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 80)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 80)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存
        print("\n" + "=" * 80)
        print("STEP 5: Saving processed data")
        print("=" * 80)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True, parents=True)
        
        # 根据dose_filter_method命名文件
        dose_method = 'method1'  # 与上面的设置保持一致
        output_file = output_dir / f"training_data_lincs2020_{dose_method}.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f, protocol=4)
        
        print(f"✓ Saved successfully!")
        print(f"   File size: {output_file.stat().st_size / (1024**2):.1f} MB")
        
        # 最终摘要
        print("\n" + "=" * 80)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 80)
        print(f"📁 Output: {output_file}")
        print(f"\n📊 Final dataset:")
        print(f"   • Samples: {len(training_data['X']):,}")
        print(f"   • Compounds: {len(training_data['compound_names']):,}")
        print(f"   • Genes: {training_data['X'].shape[1]}")
        print(f"   • Memory: {training_data['X'].nbytes / (1024**3):.2f} GB")
        
        # 与论文对比
        paper_samples = 425242
        paper_compounds = 9597
        our_samples = len(training_data['X'])
        our_compounds = len(training_data['compound_names'])
        
        print(f"\n📈 Comparison with paper:")
        print(f"   Samples: {our_samples:,} / {paper_samples:,} ({our_samples/paper_samples*100:.1f}%)")
        print(f"   Compounds: {our_compounds:,} / {paper_compounds:,} ({our_compounds/paper_compounds*100:.1f}%)")
        
        print(f"\n🎯 Ready for training!")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 80)
        print("❌ ERROR")
        print("=" * 80)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    training_data = main()

🧬 DRUGREFLECTOR DATA PREPROCESSING - Enhanced Version v2
   With intelligent dose filtering methods

🔬 LINCS 2020 Data Loader - Enhanced Version v2
Data directory: E:\科研\Models\drugreflector\datasets\LINCS2020
Dataset: Expanded CMap LINCS Resource 2020


STEP 1: Loading metadata
📖 Loading gene information...
   File: geneinfo_beta.txt
   ✓ Loaded 12,328 genes
   ✓ Columns: ['gene_id', 'gene_symbol', 'ensembl_id', 'gene_title', 'gene_type', 'src', 'feature_space']
   ✓ Using 'feature_space' column to identify landmarks
   ✓ Landmark genes: 978
   ✓ Expected: 978

   Landmark column indices (first 10): [2154 2155 2156 2157 2158 2159 2160 2161 2162 2163]
   Sample IDs: ['4154', '9710', '2263', '55256', '3162']
   Sample symbols: ['TIMELESS', 'TNIP1', 'ATF1', 'FGFR4', 'SMNDC1']

📖 Loading cell information...
   File: cellinfo_beta.txt
   ✓ Loaded 240 cell lines
   ✓ Columns: ['cell_iname', 'cellosaurus_id', 'donor_age', 'donor_age_death', 'donor_disease_age_onset', 'doubling_time', 'growth

   Computing:  83%|████████▎ | 18929/22731 [09:14<01:51, 34.00it/s]  

In [6]:
import os
from pathlib import Path

def clean_cache_files(cache_dir='./cache'):
    """删除所有缓存的 .npy 文件"""
    cache_path = Path(cache_dir)
    if cache_path.exists():
        npy_files = list(cache_path.glob('**/*.npy'))
        if npy_files:
            print(f"🗑️  Found {len(npy_files)} cache files")
            for f in npy_files:
                print(f"   Deleting: {f}")
                f.unlink()
            print("✅ Cache cleaned!")
        else:
            print("ℹ️  No cache files found")
    else:
        print(f"ℹ️  Cache directory {cache_dir} does not exist")

# 运行清理
clean_cache_files('./cache')  # 根据你的缓存目录路径修改

ℹ️  Cache directory ./cache does not exist
